In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive
import math

# --- 1. CONFIGURATION: VERIFY VALUES BELOW ---
# The path to your VWW dataset parent folder in Google Drive
BASE_DIR = '/content/drive/MyDrive/EdgeImpulse_VWW/dataset_person_vww'
IMG_SIZE = (96, 96)
BATCH_SIZE = 32
EPOCHS = 10
# -----------------------------------------------

# --- 2. MOUNT GOOGLE DRIVE ---
print("1. Mounting Google Drive...")
drive.mount('/content/drive', force_remount=True)

# --- 3. DATA PREPARATION ---
print("2. Preparing Data Generators...")

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    BASE_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',
    seed=42
)

# --- 4. MODEL DEFINITION AND TRAINING ---
print("3. Building and Training Model...")

# Define the model (must match the one you trained)
model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    MaxPooling2D((2, 2)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Re-run training (verbose=0 suppresses the epoch output)
model.fit(
    train_generator,
    epochs=EPOCHS,
    verbose=0
)
print("   (Training completed successfully. Proceeding to conversion.)")

# --- 5. TFLITE CONVERSION (FIXED) ---
print("\n4. Converting to TFLite (Quantization)...")

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Define a representative dataset generator (FIXED to use a simple counter)
def representative_data_gen(steps_to_yield=100):
    # Create a separate generator for representative data (batch_size=1)
    rep_data_generator = datagen.flow_from_directory(
        BASE_DIR,
        target_size=IMG_SIZE,
        batch_size=1,
        class_mode='binary',
        subset='training',
        seed=42
    )

    print(f"   Generating {steps_to_yield} representative data samples for quantization...")
    for i, (input_value, _) in enumerate(rep_data_generator):
        yield [input_value.astype(np.float32)]
        if i >= steps_to_yield - 1:
            break

# Configure and perform the conversion
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

tflite_model_quant = converter.convert()

# Save the TFLite model
tflite_model_path = os.path.join(BASE_DIR, 'person_detector_quantized.tflite')
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model_quant)

print(f"✅ TFLite model saved successfully to: {tflite_model_path}")
print(f"   Model size: {os.path.getsize(tflite_model_path) / 1024:.2f} KB")


# --- 6. C ARRAY CONVERSION ---
print("\n5. Converting to C Array Header...")

def convert_to_c_array(tflite_bytes, variable_name="model"):
    # Convert bytes to a C-style hex string array
    hex_array = ', '.join([f'0x{b:02x}' for b in tflite_bytes])

    # Create the C header file content
    c_code = f"""
// Auto-generated file for TFLite Micro deployment.
// Model name: {variable_name}
// Model size: {len(tflite_bytes)} bytes
const unsigned char {variable_name}[] __attribute__((aligned(16))) = {{
  {hex_array}
}};
const int {variable_name}_len = {len(tflite_bytes)};
"""
    return c_code

# Get the C array code
c_array_code = convert_to_c_array(tflite_model_quant, variable_name="person_model")

# Save the C array to a header file
header_file_path = os.path.join(BASE_DIR, 'person_model.h')
with open(header_file_path, 'w') as f:
    f.write(c_array_code)

print(f"✅ C Header file saved successfully to: {header_file_path}")

print("\n--- Pipeline Confirmed! Files are ready for your ESP32S3 project. ---")

1. Mounting Google Drive...
Mounted at /content/drive
2. Preparing Data Generators...
Found 3692 images belonging to 2 classes.
3. Building and Training Model...
   (Training completed successfully. Proceeding to conversion.)

4. Converting to TFLite (Quantization)...
Saved artifact at '/tmp/tmpjnhl62nb'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name='keras_tensor_8')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  139016640714064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139016640714832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139016640712912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139016640713296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139016640715984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139016640715216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139016640716368: TensorSp

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


Found 3692 images belonging to 2 classes.
   Generating 100 representative data samples for quantization...
✅ TFLite model saved successfully to: /content/drive/MyDrive/EdgeImpulse_VWW/dataset_person_vww/person_detector_quantized.tflite
   Model size: 979.84 KB

5. Converting to C Array Header...
✅ C Header file saved successfully to: /content/drive/MyDrive/EdgeImpulse_VWW/dataset_person_vww/person_model.h

--- Pipeline Confirmed! Files are ready for your ESP32S3 project. ---


### Model Architecture

In [ ]:
print('Model Summary:')
model.summary()

Model Summary:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 94, 94, 16)     │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 47, 47, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 45, 45, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 22, 22, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 15488)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       991,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,989,349 (11.40 MB)

 Trainable params: 996,449 (3.80 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,992,900 (7.60 MB)

### Model Training Performance

In [ ]:
print('Evaluating model on training data...')
loss, accuracy = model.evaluate(train_generator, verbose=0)
print(f"Training Loss: {loss:.4f}")
print(f"Training Accuracy: {accuracy:.4f}")

Evaluating model on training data...
Training Loss: 0.1619
Training Accuracy: 0.9445
